In [3]:

import pandas as pd
from util.BertAugmentation import BertAugmentation
from pathlib import Path


BERT_aug = BertAugmentation()
random_masking_insertion = BERT_aug.random_masking_insertion

sentence = "이것은 엄청난 테스트입니다"
ratio = 0.15
print('random_insertion_replacement:', random_masking_insertion(sentence, ratio))

import pandas as pd

# Load dataset
input_path = '../static/data/음식점_train.csv'
output_path = '../static/data/augmented_questions.csv'
data = pd.read_csv(input_path)

# Filter for QA여부 == 'q' and select the first 10 rows
data = data[data["QA여부"].str.lower() == 'q'].head(10)

data["증강된 발화문"] = data["발화문"].apply(lambda x: random_masking_insertion(x, ratio))
Path("static/data").mkdir(parents=True, exist_ok=True)
data.to_csv(output_path, index=False)

NameError: name '__file__' is not defined

In [ ]:
!pip install protobuf
!pip install transformers
!pip install torch torchvision torchaudio